In [86]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [87]:
import tensorflow as tf
from tensorflow import keras

import IPython

!pip install -U keras-tuner
import kerastuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [88]:
from tensorflow.keras.datasets import fashion_mnist

In [89]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [90]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [91]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))
  
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 
  #hP_opt = hp.Choice('', values = ['adam', '', 1e-4])
  
  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])
  
  return model

In [92]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [93]:
tuner = kerastuner.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'kerastuner') 

INFO:tensorflow:Reloading Oracle from existing project my_dir/kerastuner/oracle.json


In [94]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [96]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")



Epoch 1/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5343 - accuracy: 0.8095 - val_loss: 0.5334 - val_accuracy: 0.8101
Epoch 2/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4431 - accuracy: 0.8432 - val_loss: 0.4765 - val_accuracy: 0.8321


Epoch 1/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5503 - accuracy: 0.8116 - val_loss: 0.5051 - val_accuracy: 0.8221
Epoch 2/2
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4165 - accuracy: 0.8538 - val_loss: 0.4309 - val_accuracy: 0.8471


Epoch 1/2
1875/1875 [==============================] - 8s 5ms/step - loss: 0.5422 - accuracy: 0.8119 - val_loss: 0.4659 - val_accuracy: 0.8354
Epoch 2/2
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4294 - accuracy: 0.8458 - val_loss: 0.4557 - val_accuracy: 0.8447


Epoch 1/2
1875/1875 [==============================] - 7s 3ms/step - loss: 0.6250 - accuracy: 0.7979 - val_loss: 0.5087 - val_accuracy: 0.8252
Epoch 2/2
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4420 - accuracy: 0.8499 - val_loss: 0.4431 - val_accuracy: 0.8481


Epoch 1/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5198 - accuracy: 0.8201 - val_loss: 0.4401 - val_accuracy: 0.8438
Epoch 2/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3943 - accuracy: 0.8595 - val_loss: 0.3989 - val_accuracy: 0.8613


...

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.5291 - accuracy: 0.8127 - val_loss: 0.4723 - val_accuracy: 0.8270
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4358 - accuracy: 0.8452 - val_loss: 0.5360 - val_accuracy: 0.8182
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4124 - accuracy: 0.8533 - val_loss: 0.4667 - val_accuracy: 0.8340
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4021 - accuracy: 0.8579 - val_loss: 0.4699 - val_accuracy: 0.8417
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3946 - accuracy: 0.8597 - val_loss: 0.4685 - val_accuracy: 0.8393
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3803 - accuracy: 0.8639 - val_loss: 0.4228 - val_accuracy: 0.8553
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3732 - accuracy: 0.8661 - val_loss: 0.4565 - val_accuracy:

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4763 - accuracy: 0.8313 - val_loss: 0.4365 - val_accuracy: 0.8335
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3605 - accuracy: 0.8684 - val_loss: 0.3880 - val_accuracy: 0.8623
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3219 - accuracy: 0.8821 - val_loss: 0.3501 - val_accuracy: 0.8724
Epoch 4/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2992 - accuracy: 0.8885 - val_loss: 0.3457 - val_accuracy: 0.8767
Epoch 5/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2798 - accuracy: 0.8968 - val_loss: 0.3439 - val_accuracy: 0.8748
Epoch 6/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2693 - accuracy: 0.8997 - val_loss: 0.3401 - val_accuracy: 0.8810
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2545 - accuracy: 0.9040 - val_loss: 0.3272 - val_accuracy:

Epoch 1/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5932 - accuracy: 0.8048 - val_loss: 0.4759 - val_accuracy: 0.8369
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.4270 - accuracy: 0.8536 - val_loss: 0.4348 - val_accuracy: 0.8488
Epoch 3/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3872 - accuracy: 0.8649 - val_loss: 0.4258 - val_accuracy: 0.8488
Epoch 4/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3627 - accuracy: 0.8741 - val_loss: 0.4098 - val_accuracy: 0.8541
Epoch 5/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3421 - accuracy: 0.8805 - val_loss: 0.3826 - val_accuracy: 0.8644
Epoch 6/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3282 - accuracy: 0.8840 - val_loss: 0.3616 - val_accuracy: 0.8717
Epoch 7/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3151 - accuracy: 0.8883 - val_loss: 0.3727 - val_accura

Epoch 1/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4766 - accuracy: 0.8321 - val_loss: 0.3913 - val_accuracy: 0.8599
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3618 - accuracy: 0.8673 - val_loss: 0.3992 - val_accuracy: 0.8519
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3248 - accuracy: 0.8792 - val_loss: 0.3497 - val_accuracy: 0.8740
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3015 - accuracy: 0.8891 - val_loss: 0.3560 - val_accuracy: 0.8695
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2821 - accuracy: 0.8958 - val_loss: 0.3444 - val_accuracy: 0.8780
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2693 - accuracy: 0.9003 - val_loss: 0.3783 - val_accuracy: 0.8674
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2552 - accuracy: 0.9050 - val_loss: 0.3345 - val_accuracy:

INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 352 and the optimal learning rate for the optimizer
is 0.001.



In [97]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4801 - accuracy: 0.8295 - val_loss: 0.4145 - val_accuracy: 0.8518
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3612 - accuracy: 0.8685 - val_loss: 0.3907 - val_accuracy: 0.8605
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3235 - accuracy: 0.8814 - val_loss: 0.3689 - val_accuracy: 0.8661
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3024 - accuracy: 0.8884 - val_loss: 0.3416 - val_accuracy: 0.8753
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2830 - accuracy: 0.8964 - val_loss: 0.3521 - val_accuracy: 0.8751
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2663 - accuracy: 0.9011 - val_loss: 0.3237 - val_accuracy: 0.8845
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2528 - accuracy: 0.9062 - val_loss: 0.3316 - val_accuracy:

**Example**

In [98]:
import numpy as np
import pandas as pd

In [111]:
df=pd.read_csv('/content/gdrive/My Drive/datasets/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [112]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [113]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [114]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [115]:
import kerastuner as kt
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [116]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer', values = ['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [117]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [118]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
20/20 [==============================] - 0s 9ms/step - loss: 0.6853 - accuracy: 0.6303 - val_loss: 0.6300 - val_accuracy: 0.6818
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6189 - accuracy: 0.6661 - val_loss: 0.5836 - val_accuracy: 0.7338
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5787 - accuracy: 0.7052 - val_loss: 0.5494 - val_accuracy: 0.7338
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5497 - accuracy: 0.7134 - val_loss: 0.5244 - val_accuracy: 0.7922
Epoch 5/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5282 - accuracy: 0.7264 - val_loss: 0.5071 - val_accuracy: 0.7857


Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.6936 - accuracy: 0.5619 - val_loss: 0.7067 - val_accuracy: 0.4935
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6707 - accuracy: 0.6059 - val_loss: 0.6826 - val_accuracy: 0.5779
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6504 - accuracy: 0.6466 - val_loss: 0.6625 - val_accuracy: 0.6234
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6332 - accuracy: 0.6661 - val_loss: 0.6441 - val_accuracy: 0.6753
Epoch 5/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6177 - accuracy: 0.6922 - val_loss: 0.6292 - val_accuracy: 0.7013


Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.7166 - accuracy: 0.5668 - val_loss: 0.7251 - val_accuracy: 0.5974
Epoch 2/5
20/20 [==============================] - 0s 2ms/step - loss: 0.7166 - accuracy: 0.5668 - val_loss: 0.7251 - val_accuracy: 0.5974
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.7165 - accuracy: 0.5668 - val_loss: 0.7250 - val_accuracy: 0.5974
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.7164 - accuracy: 0.5668 - val_loss: 0.7249 - val_accuracy: 0.5974
Epoch 5/5
20/20 [==============================] - 0s 2ms/step - loss: 0.7164 - accuracy: 0.5668 - val_loss: 0.7249 - val_accuracy: 0.5974


INFO:tensorflow:Oracle triggered exit


In [119]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [120]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                288       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [121]:
model.fit(X_train,y_train,batch_size=32,epochs=10,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/10
20/20 [==============================] - 0s 8ms/step - loss: 0.5286 - accuracy: 0.7215 - val_loss: 0.5082 - val_accuracy: 0.7857
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5134 - accuracy: 0.7378 - val_loss: 0.4956 - val_accuracy: 0.7792
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5009 - accuracy: 0.7492 - val_loss: 0.4852 - val_accuracy: 0.7792
Epoch 10/10
20/20 [==============================] - 0s 2ms/step - loss: 0.4909 - accuracy: 0.7655 - val_loss: 0.4772 - val_accuracy: 0.7922


In [122]:
def build_model(hp):

  model = Sequential()

  units = hp.Int('units',min_value = 8,max_value = 128)

  model.add(Dense(units=units, activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [123]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir',
                        project_name='srk')

In [124]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.6218 - accuracy: 0.7182 - val_loss: 0.5644 - val_accuracy: 0.7922
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5687 - accuracy: 0.7410 - val_loss: 0.5314 - val_accuracy: 0.7987
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5373 - accuracy: 0.7508 - val_loss: 0.5059 - val_accuracy: 0.7922
Epoch 4/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5132 - accuracy: 0.7557 - val_loss: 0.4891 - val_accuracy: 0.7987
Epoch 5/5
20/20 [==============================] - 0s 2ms/step - loss: 0.4956 - accuracy: 0.7590 - val_loss: 0.4789 - val_accuracy: 0.7987


Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.7568 - accuracy: 0.4495 - val_loss: 0.6926 - val_accuracy: 0.5130
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.7059 - accuracy: 0.5358 - val_loss: 0.6600 - val_accuracy: 0.6104
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6663 - accuracy: 0.6026 - val_loss: 0.6329 - val_accuracy: 0.6753
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6339 - accuracy: 0.6433 - val_loss: 0.6095 - val_accuracy: 0.7208
Epoch 5/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6072 - accuracy: 0.6889 - val_loss: 0.5897 - val_accuracy: 0.7338


Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.6511 - accuracy: 0.6726 - val_loss: 0.5898 - val_accuracy: 0.7403
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5915 - accuracy: 0.7134 - val_loss: 0.5486 - val_accuracy: 0.7532
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5555 - accuracy: 0.7427 - val_loss: 0.5192 - val_accuracy: 0.7922
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5287 - accuracy: 0.7508 - val_loss: 0.4964 - val_accuracy: 0.7922
Epoch 5/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5085 - accuracy: 0.7590 - val_loss: 0.4823 - val_accuracy: 0.7922


Epoch 1/5
20/20 [==============================] - 0s 8ms/step - loss: 0.6359 - accuracy: 0.6401 - val_loss: 0.5854 - val_accuracy: 0.6623
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5679 - accuracy: 0.7101 - val_loss: 0.5426 - val_accuracy: 0.7403
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5318 - accuracy: 0.7459 - val_loss: 0.5154 - val_accuracy: 0.7597
Epoch 4/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5073 - accuracy: 0.7590 - val_loss: 0.4960 - val_accuracy: 0.7662
Epoch 5/5
20/20 [==============================] - 0s 3ms/step - loss: 0.4904 - accuracy: 0.7655 - val_loss: 0.4863 - val_accuracy: 0.7532


Epoch 1/5
20/20 [==============================] - 0s 9ms/step - loss: 0.6508 - accuracy: 0.6498 - val_loss: 0.6326 - val_accuracy: 0.6883
Epoch 2/5
20/20 [==============================] - 0s 3ms/step - loss: 0.6103 - accuracy: 0.6792 - val_loss: 0.6019 - val_accuracy: 0.7532
Epoch 3/5
20/20 [==============================] - 0s 3ms/step - loss: 0.5821 - accuracy: 0.6938 - val_loss: 0.5771 - val_accuracy: 0.7857
Epoch 4/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5581 - accuracy: 0.7052 - val_loss: 0.5564 - val_accuracy: 0.7792
Epoch 5/5
20/20 [==============================] - 0s 2ms/step - loss: 0.5381 - accuracy: 0.7296 - val_loss: 0.5396 - val_accuracy: 0.7792


INFO:tensorflow:Oracle triggered exit


In [125]:
tuner.get_best_hyperparameters()[0].values

{'units': 62}

In [126]:
model = tuner.get_best_models(num_models=1)[0]

In [127]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5381 - accuracy: 0.7508
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5142 - accuracy: 0.7524
Epoch 9/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4978 - accuracy: 0.7557
Epoch 10/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4851 - accuracy: 0.7638
Epoch 11/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4757 - accuracy: 0.7720
Epoch 12/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4701 - accuracy: 0.7704
Epoch 13/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4661 - accuracy: 0.7818
Epoch 14/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4637 - accuracy: 0.7687
Epoch 15/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4594 - accuracy: 0.7769
Epoch 16/100
20/20 [==============================] - 0s 2ms/step - loss: 0.4573 - accuracy: 0.7769
Epo

In [128]:
def build_model1(hp):

  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [129]:
def build_model2(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:

      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128,step=8),activation= hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:

      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128,step=8),activation= hp.Choice('activation' + str(i), values=['relu','tanh','sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
  return model